In [24]:
import os
import psutil
import geopandas as gpd
from matrixconverters.read_ptv import ReadPTVMatrix
import xarray as xr

In [2]:
os.chdir("..")

In [3]:
from params.project_params import ENCODING_CP1252, CRS_EPSG_ID_WGS84
from scripts.functions import get_gdf_mobility_stations, get_gdf_mobility_stations_with_npvm_zone, get_skim

In [4]:
path_to_npvm_zones_shp = os.path.join('data', 'Verkehrszonen_Schweiz_NPVM_2017_shp.zip')

In [5]:
path_mobility_stations = os.path.join('data', 'mobility-stationen-und-fahrzeuge-schweiz.csv')

In [6]:
skim_paths = {
    'DWV_2017_ÖV_Reisezeit_CH_binär.mtx': 'pt_jt.nc',
    '144_NTR_(OEV).mtx': 'pt_nt.nc',
    'DWV_2017_ÖV_Distanz_CH_binär.mtx': 'pt_dist.nc',
    'DWV_2017_Strasse_Reisezeit_CH_binär.mtx': 'road_jt.nc',
    'DWV_2017_Strasse_Distanz_CH_binär.mtx': 'road_dist.nc'
}

In [7]:
process = psutil.Process()

In [8]:
def print_memory_usage():
    print(process.memory_info().rss / 1024 ** 2)

In [9]:
print_memory_usage()

115.58984375


In [10]:
gdf_npvm_zones = gpd.read_file(path_to_npvm_zones_shp, encoding=ENCODING_CP1252).to_crs(CRS_EPSG_ID_WGS84)
print_memory_usage()

204.39453125


In [11]:
gdf_mobility_stations = get_gdf_mobility_stations(path_mobility_stations)
print_memory_usage()

208.8125


In [12]:
gdf_mobility_stations_with_npvm_zone = get_gdf_mobility_stations_with_npvm_zone(gdf_mobility_stations, gdf_npvm_zones)
print_memory_usage()

213.359375


In [19]:
npvm_zones_with_mobility_station = list(set(gdf_mobility_stations_with_npvm_zone['ID'].values))

In [22]:
def read_skim_filter_and_write(path_old, path_new):
    print_memory_usage()
    print(path_old, path_new)
    skim = ReadPTVMatrix(os.path.join('data', path_old))
    skim_filtered = skim.sel(destinations=npvm_zones_with_mobility_station)
    skim_filtered.to_netcdf(os.path.join('data', path_new))
    print_memory_usage()

In [23]:
for k, v in skim_paths.items():
    read_skim_filter_and_write(k, v)

225.4765625
DWV_2017_ÖV_Reisezeit_CH_binär.mtx pt_jt.nc
799.265625
233.86328125
144_NTR_(OEV).mtx pt_nt.nc
901.77734375
234.71484375
DWV_2017_ÖV_Distanz_CH_binär.mtx pt_dist.nc
800.1328125
234.73046875
DWV_2017_Strasse_Reisezeit_CH_binär.mtx road_jt.nc
800.140625
234.73828125
DWV_2017_Strasse_Distanz_CH_binär.mtx road_dist.nc
800.140625


In [27]:
print_memory_usage()
skims = []
for v in skim_paths.values():
    skims += [xr.open_dataset(os.path.join('data', v))]
print_memory_usage()

236.625
244.17578125


In [30]:
skims[3]

<xarray.Dataset>
Dimensions:       (zone_no: 7978, zone_no2: 7978, origins: 7978,
                   destinations: 1311)
Coordinates:
  * zone_no       (zone_no) int32 101001 201001 201002 ... 710101001 730101001
  * zone_no2      (zone_no2) int32 101001 201001 201002 ... 710101001 730101001
  * origins       (origins) int32 101001 201001 201002 ... 710101001 730101001
  * destinations  (destinations) int32 575701002 15401001 ... 163201010 9601011
Data variables:
    matrix        (origins, destinations) float64 ...
    zone_name     (zone_no) object ...
    zone_names2   (zone_no2) object ...
Attributes:
    fn:                   data\DWV_2017_Strasse_Reisezeit_CH_binär.mtx
    ZeitVon:              nan
    ZeitBis:              nan
    Faktor:               1.0
    VMAktKennung:         0
    AnzBezeichnerlisten:  1
    roundproc:            1
    diagsum:              59946.9967931197

In [31]:
npvm_zone_per_mobility_station = {}
for e in gdf_mobility_stations_with_npvm_zone[['Stationsnummer', 'ID']].to_dict('records'):
    mob_stat_nr = e['Stationsnummer']
    npvm_id = e['ID']
    if mob_stat_nr in npvm_zone_per_mobility_station:
        if npvm_id != npvm_zone_per_mobility_station[mob_stat_nr]:
            raise ValueError(f'npvm zone for mobility station {mob_stat_nr} not unique')
    else:
        npvm_zone_per_mobility_station[mob_stat_nr] = npvm_id

In [38]:
%%time
orig_npvm_zone = 201001
data_per_mob_st_id = {}
for mob_st_id, npvm_zone_id in npvm_zone_per_mobility_station.items():
    data = []
    for s in skims:
        data += [s.sel(origins=orig_npvm_zone, destinations=npvm_zone_id)]
    data_per_mob_st_id[mob_st_id, npvm_zone_id] = data
        


CPU times: total: 2.02 s
Wall time: 2.02 s


In [46]:
for e in s.sel(origins=orig_npvm_zone):
    print(e.)
    break

matrix


In [57]:
asdf = s.sel(origins=orig_npvm_zone)

In [70]:
for e in asdf:
    print(e.values)
    break

AttributeError: 'str' object has no attribute 'values'